In [1]:
import requests
import pandas as pd
from mainnet_launch.constants import AUTO_ETH
from web3 import Web3"


def paginate_query(api_url: str, query: str, variables: dict, data_col: str) -> list[dict]:
    """
    Helper to page through a GraphQL connection using `first`/`skip`.

    :param api_url: GraphQL endpoint
    :param query: The GraphQL query string, expecting $first and $skip variables
    :param variables: Base variables (e.g. {"autoEthAddress": "..."}).
                      first/skip will be merged in each loop.
    :param batch_size: Number of items to fetch per request.
    :return: List of result dicts
    """

    all_records = []
    skip = 0

    while True:
        vars_with_pagination = {**variables, "first": 500, "skip": skip}
        resp = requests.post(api_url, json={"query": query, "variables": vars_with_pagination})
        resp.raise_for_status()
        batch = resp.json()["data"][data_col]

        if not batch:
            break

        all_records.extend(batch)
        skip += 500

    return all_records


def _get_subgraph_api(chain):
    if chain == "eth":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-eth-mainnet/api"
    elif chain == "base":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-base-mainnet/api"
    else:
        raise ValueError("bad chain", chain)

    return api_url


def fetch_autopool_rebalance_events_from_subgraph(autopool_eth_addr: str, chain: str) -> list[dict]:
    """
    Fetches all AutopoolRebalances entries for the given autopool.
    """
    subgraph_url = _get_subgraph_api(chain)

    query = """
    query($autoEthAddress: String!, $first: Int!, $skip: Int!) {
      autopoolRebalances(
        first: $first,
        skip: $skip,
        orderBy: id,
        orderDirection: desc,
        where: { autopool: $autoEthAddress }
      ) {
        transactionHash
        timestamp
        blockNumber

        tokenInAmount
        tokenOut {
        id
        decimals
        
        }        
        destinationInAddress

        tokenIn {
        id
        decimals
        
        }   

        tokenOutAmount
        destinationOutAddress
        
      }
    }
    """

    # Adjust `first` as needed; paginate_query will loop over skip increments of `first`
    df = pd.DataFrame.from_records(
        paginate_query(
            subgraph_url,
            query,
            variables={"autoEthAddress": autopool_eth_addr.lower(), "first": 1000, "skip": 0},
            data_col="autopoolRebalances",
        )
    )

    df["tokenInAddress"] = df["tokenIn"].apply(lambda x: x["id"])
    df["tokenOutAddress"] = df["tokenOut"].apply(lambda x: x["id"])

    df["tokenOutAmount"] = df.apply(
        lambda row: int(row["tokenOutAmount"]) / (10 ** int(row["tokenOut"]["decimals"])), axis=1
    )
    df["tokenInAmount"] = df.apply(
        lambda row: int(row["tokenInAmount"]) / (10 ** int(row["tokenIn"]["decimals"])), axis=1
    )

    df["blockNumber"] = df["blockNumber"].astype(int)

    df["destinationInAddress"] = df["destinationInAddress"].apply(lambda x: Web3.toChecksumAddress(x))
    df["tokenInAddress"] = df["tokenInAddress"].apply(lambda x: Web3.toChecksumAddress(x))

    df["destinationOutAddress"] = df["destinationOutAddress"].apply(lambda x: Web3.toChecksumAddress(x))
    df["tokenOutAddress"] = df["tokenOutAddress"].apply(lambda x: Web3.toChecksumAddress(x))

    return df


rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph("0xa7569A44f348d3D70d8ad5889e50F78E33d80D35", "eth")
rebalance_event_df

SyntaxError: unterminated string literal (detected at line 4) (2459460644.py, line 4)

In [ ]:
from mainnet_launch.database.schema.full import DestinationStates, Destinations, AutopoolStates, RebalancePlans
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
)
from mainnet_launch.database.schema.ensure_tables_are_current.update_destinations_states_table import (
    _add_new_destination_states_to_db,
)

from mainnet_launch.database.schema.ensure_tables_are_current.update_autopool_states import (
    _add_new_autopool_states_to_db,
)

_add_new_destination_states_to_db(rebalance_event_df["blockNumber"], AUTO_ETH.chain)
_add_new_autopool_states_to_db(rebalance_event_df["blockNumber"], AUTO_ETH.chain)


destination_states_df = merge_tables_as_df(
    [
        TableSelector(DestinationStates, [DestinationStates.block, DestinationStates.lp_token_spot_price]),
        TableSelector(
            Destinations,
            [Destinations.destination_vault_address, Destinations.underlying_symbol],
            join_on=(DestinationStates.chain_id == DestinationStates.chain_id)
            & (Destinations.destination_vault_address == DestinationStates.destination_vault_address),
        ),
        TableSelector(
            AutopoolStates,
            [AutopoolStates.total_nav],
            join_on=(AutopoolStates.chain_id == DestinationStates.chain_id)
            & (AutopoolStates.block == DestinationStates.block),
        ),
    ],
    where_clause=AutopoolStates.autopool_vault_address == "0xa7569A44f348d3D70d8ad5889e50F78E33d80D35",
)
destination_states_df

2025-05-05 16:35:46,889 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-05 16:35:46,890 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 16:35:47,102 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-05 16:35:47,105 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 16:35:47,296 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-05 16:35:47,298 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 16:35:47,484 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 16:35:47,485 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM destination_states
            WHERE destination_states.chain_id = 1
        
2025-05-05 16:35:47,486 INFO sqlalchemy.engine.Engine [generated in 0.00089s] {}
2025-05-05 16:35:49,331 INFO sqlalchemy.engine.Engine COMMIT
2025-05-05 16:35:49,466 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 16:35:49,466 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM bloc

,block,lp_token_spot_price,destination_vault_address,underlying_symbol,total_nav
0,22253035,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
1,22254021,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
2,22254610,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
3,22255051,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
4,22255509,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
...,...,...,...,...,...
12679,22420454,1.000000,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH,0.000029
12680,22420454,1.000000,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,balETH,0.000029
12681,22420454,1.000000,0xE800e3760FC20aA98c5df6A9816147f190455AF3,autoLRT,0.000029
12682,22420454,1.000000,0x35911af1B570E26f668905595dEd133D01CD3E5a,dineroETH,0.000029


In [ ]:
# this works

In [17]:
price_df = destination_states_df[["block", "destination_vault_address", "lp_token_spot_price"]]

df1 = rebalance_event_df.merge(
    price_df,
    left_on=["blockNumber", "destinationInAddress"],
    right_on=["block", "destination_vault_address"],
    how="left",
)
# rename & drop the merge‑helper cols
df1.rename(columns={"lp_token_spot_price": "lp_token_in_spot_price"}, inplace=True)
df1.drop(columns=["block", "destination_vault_address"], inplace=True)


df1 = df1.merge(
    price_df,
    left_on=["blockNumber", "destinationOutAddress"],
    right_on=["block", "destination_vault_address"],
    how="left",
)
df1.rename(columns={"lp_token_spot_price": "lp_token_out_spot_price"}, inplace=True)
df1.drop(columns=["block", "destination_vault_address"], inplace=True)

df1["spot_value_in"] = df1["tokenInAmount"] * df1["lp_token_in_spot_price"]
df1["spot_value_out"] = df1["tokenOutAmount"] * df1["lp_token_out_spot_price"]


df1["swap_cost"] = df1["spot_value_out"] - df1["spot_value_in"]
df1["slippage"] = (100 * (df1["spot_value_out"] - df1["spot_value_in"])) / df1["spot_value_out"]

block_and_nav = destination_states_df[["block", "total_nav"]].drop_duplicates()
df1 = pd.merge(df1, block_and_nav, left_on="blockNumber", right_on=["block"])
df1

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenOut,destinationInAddress,tokenIn,tokenOutAmount,destinationOutAddress,tokenInAddress,tokenOutAddress,lp_token_in_spot_price,lp_token_out_spot_price,spot_value_in,spot_value_out,swap_cost,slippage,block,total_nav
0,0xff8915d0bd9054e7678c4366ae8fd5a0a3cce80943be...,1744988327,22296632,148881.150087,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.006900,1.000000,149908.430023,150000.000000,91.569977,0.061047,22296632,0.000018
1,0xff3d22a3564c8121f28d7d47feffb8251785ad2c48e4...,1744997471,22297389,148885.474474,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.006902,1.000000,149913.082019,150000.000000,86.917981,0.057945,22297389,0.000022
2,0xfecf659a8d1c06c45010afc47fcefc6405d906a85e84...,1746235043,22399927,320651.575857,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xC5c95fCad37E466E25e6ecA1977bbF75C0E1004a,{'id': '0xbeef01735c132ada46aa9aa4c54623caa92a...,347222.682150,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xBEEF01735c132Ada46AA9aA4c54623cAA92A64CB,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.082884,1.000000,347228.461070,347222.682150,-5.778920,-0.001664,22399927,0.000028
3,0xfca3e4ad4e6296b2444ed28a8012cda8694ef5d23ce9...,1746116255,22390138,148075.994800,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7876F91BB22148345b3De16af9448081E9853830,{'id': '0x9fb7b4477576fe5b32be4c1843afb1e55f25...,170000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.148038,1.000000,169996.868918,170000.000000,3.131082,0.001842,22390138,0.000025
4,0xfc37985d3c7182369be30f9dbe69409a8b05d008bde2...,1745111003,22306807,437134.294459,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61,{'id': '0x5c20b550819128074fd538edf79791733cce...,500000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x5C20B550819128074FD538Edf79791733ccEdd18,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.143221,1.000000,499741.105246,500000.000000,258.894754,0.051779,22306807,0.000022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,0x02a5528289efe28718bed54e9c5b0772f16eba69d609...,1744695875,22272381,76347.504659,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,77679.210200,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.017084,1.000000,77651.825428,77679.210200,27.384772,0.035254,22272381,0.000010
266,0x02065d1b16c7c693afe6358f38647b921a095d6876ec...,1745911055,22373171,26611.724622,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,27068.157900,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.017144,1.000000,27067.956028,27068.157900,0.201872,0.000746,22373171,0.000024
267,0x012e26a2633c10388ffc3ee1a2a95c9569b54ff4f918...,1744556327,22260800,140053.102440,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,0xC099899d0278CE83976218Cbe58D01dD382dcA32,{'id': '0x57064f49ad7123c92560882a45518374ad98...,149036.479162,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0x57064F49Ad7123C92560882a45518374ad982e85,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,1.071202,1.006465,150025.163440,150000.000000,-25.163440,-0.016776,22260800,0.000008
268,0x00ff

In [30]:
avg_nav = df1.resample("1d")["total_nav"].mean()
total_swap_cost = df1.resample("1d")["swap_cost"].sum()
total_swap_cost

timestamp
2025-04-03 00:00:00+00:00       0.025546
2025-04-04 00:00:00+00:00       6.480371
2025-04-05 00:00:00+00:00       0.636527
2025-04-06 00:00:00+00:00       0.000000
2025-04-07 00:00:00+00:00       5.417236
2025-04-08 00:00:00+00:00      17.599315
2025-04-09 00:00:00+00:00     135.763929
2025-04-10 00:00:00+00:00    1583.027775
2025-04-11 00:00:00+00:00    1097.118274
2025-04-12 00:00:00+00:00     472.475559
2025-04-13 00:00:00+00:00     -27.462960
2025-04-14 00:00:00+00:00     371.435712
2025-04-15 00:00:00+00:00     472.194708
2025-04-16 00:00:00+00:00     213.944928
2025-04-17 00:00:00+00:00     829.652319
2025-04-18 00:00:00+00:00     638.428830
2025-04-19 00:00:00+00:00     278.934259
2025-04-20 00:00:00+00:00    2363.807435
2025-04-21 00:00:00+00:00     418.394710
2025-04-22 00:00:00+00:00      40.957225
2025-04-23 00:00:00+00:00     143.647117
2025-04-24 00:00:00+00:00   -1146.452351
2025-04-25 00:00:00+00:00      35.155903
2025-04-26 00:00:00+00:00     106.825381
2025-0

In [33]:
avg_nav = df1.resample("1d")["total_nav"].mean() * 1e12
total_swap_cost = df1.resample("1d")["swap_cost"].sum()
import plotly.express as px

percent_of_swap_cost_per_day = 100 * (total_swap_cost) / avg_nav
px.bar(percent_of_swap_cost_per_day)

In [23]:
df1.tail(1).T

timestamp,2025-04-15 00:55:47+00:00
transactionHash,0x0000f50368b9d54cf50696552d1adeb3d070298ee8a7...
timestamp,1744678547
blockNumber,22270944
tokenInAmount,196495.452853
tokenOut,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...
destinationInAddress,0x7583b1589aDD33320366A48A92794D77763FAE9e
tokenIn,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...
tokenOutAmount,199999.999999
destinationOutAddress,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
tokenInAddress,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4


In [22]:
df1.index = pd.to_datetime(df1["timestamp"].astype(int), unit="s", utc=True)

df1.sort_index().to_csv("autoUSD_slippage_with_nav.csv")

In [ ]:

import plotly .express as px



df1.sort_values(')

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenOut,destinationInAddress,tokenIn,tokenOutAmount,destinationOutAddress,tokenInAddress,tokenOutAddress,lp_token_in_spot_price,lp_token_out_spot_price,spot_value_in,spot_value_out,slippage
240,0x1b154344809534e52c7b7293056c2ec63e61ebffcd2d...,1746330335,22407809,5474.504203,{'id': '0xbeef01735c132ada46aa9aa4c54623caa92a...,0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D,{'id': '0x4dece678ceceb27446b35c672dc7d61f30ba...,5021.109138,0xC5c95fCad37E466E25e6ecA1977bbF75C0E1004a,0x4DEcE678ceceb27446b35C672dC7d61F30bAD69E,0xBEEF01735c132Ada46AA9aA4c54623cAA92A64CB,1.018368,1.083027,5575.059896,5437.996766,-2.520471
151,0x6ab3f09e6a2ec4ef83c2dbd851f3c92a10f8fd85a611...,1745457407,22335534,148393.790366,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D,{'id': '0x4dece678ceceb27446b35c672dc7d61f30ba...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4DEcE678ceceb27446b35C672dC7d61F30bAD69E,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.018319,1.000000,151112.216211,150000.000000,-0.741477
24,0xeb1d3f3839174cf5b72ed6cd395bba5d40fd6e759169...,1745813843,22365119,147615.747990,{'id': '0x9fb7b4477576fe5b32be4c1843afb1e55f25...,0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D,{'id': '0x4dece678ceceb27446b35c672dc7d61f30ba...,130683.763587,0x7876F91BB22148345b3De16af9448081E9853830,0x4DEcE678ceceb27446b35C672dC7d61F30bAD69E,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,1.018345,1.147809,150323.758887,149999.999999,-0.215839
65,0xb4ae4eee0be0b8522433086172131818c1d42868fb95...,1746181031,22395473,72565.917676,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,73720.267300,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.017142,1.000000,73809.842637,73720.267300,-0.121507
47,0xd0563c451348fa77eea46fa4b17aa32064713f6a0547...,1745897483,22372047,39892.996621,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x0f170e37E5E0C617148517B831A49292c38c363d,{'id': '0xdd0f28e19c1780eb6396170735d45153d261...,42760.785575,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xdd0f28e19C1780eb6396170735D45153D261490d,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.072742,1.000000,42794.892981,42760.785575,-0.079763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0x998a06402a2d42b4c5b23f2a9089378841c89833bace...,1745943479,22375858,42626.221887,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,43399.473650,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.017219,1.000000,43360.202802,43399.473650,0.090487
33,0xe59412dfb0dad3fbb1a9b9deecc20a1bed4e5da0ced6...,1745207099,22314769,134365.159686,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xd7900d87069C815a299bdA7aFDcd7eEe98fe4b6c,{'id': '0x5b03cccab7ba3010fa5cad23746cbf079493...,135304.324490,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x5B03CcCAb7BA3010fA5CAd23746cbf0794938e96,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.006077,1.000000,135181.696762,135304.324490,0.090631
145,0x6eaeb51ad22b87b0ec0f1241c051721ff44258f6c028...,1746483407,22420454,36805.316815,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,37465.624500,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.016991,1.000000,37430.675953,37465.624500,0.093282
259,0x0619a8602841faf599a0b316e83f404fbf3a7bc0c3a1...,1744501463,22256246,172726.439342,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,175803.150499,0xa7569A44f348d3D70d8ad5889e50F78E3

In [5]:
df1[
    [
        "transactionHash",
        "timestamp",
        "blockNumber",
        "tokenInAmount",
        "destinationInAddress",
        "tokenOutAmount",
        "destinationOutAddress",
        "tokenInAddress",
        "tokenOutAddress",
        "lp_token_in_spot_price",
        "lp_token_out_spot_price",
        "spot_value_in",
        "spot_value_out",
        "slippage",
    ]
].to_csv("autoUSD_slippage.csv")

In [6]:
df1["slippage"] < 0

0      False
1      False
2       True
3      False
4      False
       ...  
265    False
266    False
267     True
268    False
269    False
Name: slippage, Length: 270, dtype: bool

In [7]:
df1["slippage"]

0      0.061047
1      0.057945
2     -0.001664
3      0.001842
4      0.051779
         ...   
265    0.035254
266    0.000746
267   -0.016776
268    0.003273
269    0.073613
Name: slippage, Length: 270, dtype: float64

In [24]:
px.histogram(df1["swap_cost"], title="autoUSD slippage")

In [9]:
destination_states_df

,block,lp_token_spot_price,destination_vault_address,underlying_symbol,total_nav
0,22253035,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
1,22254021,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
2,22254610,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
3,22255051,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
4,22255509,1.002066,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,OETHCRV-f,0.000008
...,...,...,...,...,...
12679,22420454,1.000000,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH,0.000029
12680,22420454,1.000000,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,balETH,0.000029
12681,22420454,1.000000,0xE800e3760FC20aA98c5df6A9816147f190455AF3,autoLRT,0.000029
12682,22420454,1.000000,0x35911af1B570E26f668905595dEd133D01CD3E5a,dineroETH,0.000029


In [10]:
destination_states_df[destination_states_df["underlying"] == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"]

KeyError: 'underlying'

In [ ]:
df1["tokenOutAddress"].value_counts()

In [ ]:
destination_states_df

In [ ]:
rebalance_event_df

In [ ]:
rebalance_event_df[["blockNumber", "destinationInAddress", "tokenInAddress"]]

In [ ]:
destination_states_df

In [ ]:
destination_states_df[["block", "destination_vault_address", "underlying"]]

In [ ]:
destination_states_df = destination_states_df.drop_duplicates()

In [ ]:
rebalance_event_df.head(1).T

In [ ]:
# 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48

In [ ]:
token_value_df[
    (token_value_df["block"] == 22296632)
    & (token_value_df["destination_vault_address"] == "0xa7569A44f348d3D70d8ad5889e50F78E33d80D35")
    & (token_value_df["token_address"] == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48")
]

In [ ]:
rebalance_event_df

In [ ]:
destination_states_df["underlying_symbol"].value_counts()